# Lucas-Kanade Optical Flow

In [13]:
import cv2
import numpy as np

# Open video file
cap = cv2.VideoCapture('/kaggle/input/sample-videos-detecting-and-matching-objs-1/sample_video_bird (5).mp4')
# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get the frame width and height for the video writer
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object to save the output video
output_filename = '/kaggle/working/output_optical_flow_4.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_filename, fourcc, 20.0, (frame_width, frame_height))

# Get the first frame
ok, frame = cap.read()
if not ok:
    print("Error: Could not read video frame.")
    cap.release()
    out.release()
    exit()

# Parameters for Shi-Tomasi corner detection
parameters_shitomasi = dict(maxCorners=100, qualityLevel=0.3, minDistance=7)

# Convert to grayscale
frame_gray_init = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Detect object corners / edges from initial frame
edges = cv2.goodFeaturesToTrack(frame_gray_init, mask=None, **parameters_shitomasi)

# Create a black canvas the size of the initial frame
canvas = np.zeros_like(frame)
# Create random colors for visualization
colours = np.random.randint(0, 255, (100, 3))

while True:
    # Get the next frame
    ok, frame = cap.read()
    if not ok:
        print("[INFO] End of file reached")
        break
    
    # Prepare grayscale image
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Update object corners by comparing with found edges in initial frame
    update_edges, status, errors = cv2.calcOpticalFlowPyrLK(frame_gray_init, frame_gray, edges, None)

    # Only update edges if successfully tracked
    new_edges = update_edges[status == 1]
    old_edges = edges[status == 1]

    # Draw the tracks
    for i, (new, old) in enumerate(zip(new_edges, old_edges)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(canvas, (int(a), int(b)), (int(c), int(d)), colours[i].tolist(), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, colours[i].tolist(), -1)

    result = cv2.add(frame, mask)

    # Write the frame to the output video
    out.write(result)
    
    # Update for the next loop
    frame_gray_init = frame_gray.copy()
    edges = new_edges.reshape(-1, 1, 2)

# Release the video capture and writer objects
cap.release()
out.release()
print("Video saved successfully.")


[INFO] End of file reached
Video saved successfully.


# Dense Optical Flow

In [18]:
import numpy as np
import cv2 as cv

# Open the video file
cap = cv.VideoCapture("/kaggle/input/sample-videos-detecting-and-matching-objs-1/sample_video_human (7).mp4")

# Get the video frame width, height, and frames per second (fps)
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
output_filename = '/kaggle/working/output_optical_flow_7.avi'
fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

ret, frame1 = cap.read()
if not ret:
    print("Error: Unable to read the first frame.")
    cap.release()
    out.release()
    exit()

prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

while True:
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break

    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)

    # Write the processed frame to the video
    out.write(bgr)

    # For debugging or verification in environments where cv.imshow is not supported
    # plt.imshow(cv.cvtColor(bgr, cv.COLOR_BGR2RGB))
    # plt.show()

    # Update the previous frame and previous points
    prvs = next

# Release the video capture and writer objects
cap.release()
out.release()

print("Video saved successfully at:", output_filename)


No frames grabbed!
Video saved successfully at: /kaggle/working/output_optical_flow_7.avi
